In [22]:
import json
import os
import pandas as pd
import re

In [42]:
## these functionalities is made into a module

In [ ]:
#Process:
## Leverage the hgvs module
## create randpm c. variants
## map to p. variants
## write out to tsv files

In [23]:
os.chdir('/mnt/data/hong/courses/varianceAnalysis/')
with open("data/ADA_uniprot.json", "r") as file:
    uniprot = file.read()
    ins = re.findall(r'(p\.[A-Z]{1}[a-z]+\d+_[A-Z]{1}[a-z]+\d+ins[A-Z]{1}[a-z]+)"',uniprot)
    deletions = re.findall(r'(p\.[A-Z]{1}[a-z]+\d+del)',uniprot)

The following block is to generate random p. insertions and deletions  
and is not necessary for the assignment

In [7]:
one2all ={'A': ('A', 'ALA', 'alanine'),
    'R': ('R', 'ARG', 'arginine'),
    'N': ('N', 'ASN', 'asparagine'),
    'D': ('D', 'ASP', 'aspartic acid'),
    'C': ('C', 'CYS', 'cysteine'),
    'Q': ('Q', 'GLN', 'glutamine'),
    'E': ('E', 'GLU', 'glutamic acid'),
    'G': ('G', 'GLY', 'glycine'),
    'H': ('H', 'HIS', 'histidine'),
    'I': ('I', 'ILE', 'isoleucine'),
    'L': ('L', 'LEU', 'leucine'),
    'K': ('K', 'LYS', 'lysine'),
    'M': ('M', 'MET', 'methionine'),
    'F': ('F', 'PHE', 'phenylalanine'),
    'P': ('P', 'PRO', 'proline'),
    'S': ('S', 'SER', 'serine'),
    'T': ('T', 'THR', 'threonine'),
    'W': ('W', 'TRP', 'tryptophan'),
    'Y': ('Y', 'TYR', 'tyrosine'),
    'V': ('V', 'VAL', 'valine'),
    'X': ('X', 'GLX', 'glutaminx'),
    'Z': ('Z', 'GLI', 'glycine'),
    'J': ('J', 'NLE', 'norleucine'),
    'U': ('U', 'CYC', 'cysteinc')
}

In [6]:
## read in fa
with open("data/ADA.fa", "r") as file:
    fa = file.read()
    fa = fa.split('\n')
    fa = [i for i in fa if i != '']
    fa = fa[1:]
    fa = ''.join(fa)
    fa = list(fa)

In [15]:
## As I didn't find a good reference for insertion or deletoin frequency, 
## and the question of this assignment has nothing to do with the frequency
## but just the functional predictoin "when" it happens
## So these indels are created by random
## I will create 9 random insertions and 9 random deletions
import random
random.seed(404)
def mk_ins(fa, random_ins, ins):
    for i in random_ins:
        aa1 = one2all[fa[i]][1]
        aa2 = one2all[fa[i+1]][1]
        aai = random.choice(list(one2all.keys()))
        insi = f'p.{aa1}{i}_{aa2}{i+1}ins{aai}'
        ins.append(insi)
    return ins

## generate 18 random integers in the range of length of fa

random_ints = random.sample(range(0, len(fa)), 18)
random_ins = random_ints[:9]
random_del = random_ints[9:]
ins_generate = mk_ins(fa, random_ins, ins)

In [17]:
def mk_del(fa, random_del, deletions):
    for i in random_del:
        aa1 = one2all[fa[i]][1]
        deli = f'p.{aa1}{i}del'
        deletions.append(deli)
    return deletions

In [19]:
deletions_generate = mk_del(fa, random_del, deletions)

In [21]:
## write out ins_generate and deletions_generate
## to tsv files
df_ins = pd.DataFrame(ins_generate, columns=['ins'])
df_del = pd.DataFrame(deletions_generate, columns=['del'])
df_ins.to_csv('data/ins.tsv', sep='\t', index=False)
df_del.to_csv('data/del.tsv', sep='\t', index=False)

In [ ]:

import pyhgvs as hgvs
from pyhgvs.utils import read_transcripts
from pyfaidx import Fasta

# Read genome sequence using pyfaidx.
genome = Fasta('/mnt/data/hong/reference/Homo_sapiens/NCBI/GRCh38/Sequence/WholeGenomeFasta/genome.fa')

# Read RefSeq transcripts into a python dict.
with open('/mnt/data/hong/reference/Homo_sapiens/refseq/hg38_all.gtf') as infile:
    transcripts = read_transcripts(infile)

# Provide a callback for fetching a transcript by its name.
def get_transcript(name):
    return transcripts.get(name)

In [39]:
transcripts['NM_000022']

In [28]:
## get codon frequency
## read in data/NG_007385.1.fa
## skip firt line
with open('data/NG_007385.1.fa') as fi:
    coding_seq = ''.join(''.join(fi.readlines()[1:]).split('\n'))

In [ ]:
codon = [coding_seq[i:i+3] for i in range(0,len(coding_seq), 3)]

In [40]:
# Parse the HGVS name into genomic coordinates and alleles.
## create HGVS names
## 'NG_007385.1':c.1234_1235insACGT
## https://databases.lovd.nl/shared/refseq/ADA_NM_000022.2_codingDNA.html
ng = 'NM_000022.2'
import random
random.seed(404)
def mk_cins():
    i = 1
    while True:
        start = random.sample(range(0, 1089), 1)[0]
        end = start +1
        ins = random.sample(codon, 1)[0]
        ins_hgvs = f'{ng}:c.{start}_{end}ins{ins}'
        yield ins_hgvs, i
        i += 1
chroms, offsets, refs, alts = [], [], [], []
for ins_names, num in mk_cins():
    if num > 9:
        break
    print(ins_names)
    chrom, offset, ref, alt = hgvs.parse_hgvs_name(ins_names, genome, get_transcript=get_transcript, lazy=True)
    chroms.append(chrom)
    offsets.append(offset)
    refs.append(ref)
    alts.append(alt)

NM_000022.2:c.179_180insTAT
NM_000022.2:c.6_7insTGG
NM_000022.2:c.191_192insTGG
NM_000022.2:c.195_196insCAA
NM_000022.2:c.274_275insCTC
NM_000022.2:c.204_205insGCT
NM_000022.2:c.983_984insGTA
NM_000022.2:c.15_16insGGG
NM_000022.2:c.376_377insTCT
